# GPU

首先是如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）。

我们先看看如何使用单个NVIDIA GPU进行计算。
首先，确保你至少安装了一个NVIDIA GPU。
然后，下载[NVIDIA驱动和CUDA](https://developer.nvidia.com/cuda-downloads)
并按照提示设置适当的路径。
当这些准备工作完成，就可以使用`nvidia-smi`命令来(**查看显卡信息。**)


In [1]:
!nvidia-smi

Fri Jan 14 23:34:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:07:00.0  On |                  N/A |
|  0%   41C    P8    27W / 400W |   1201MiB / 12288MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

要运行此部分中的程序，至少需要两个GPU。
注意，对于大多数桌面计算机来说，这可能是奢侈的，但在云中很容易获得。
例如，你可以使用AWS EC2的多GPU实例。
本书的其他章节大都不需要多个GPU，
而本节只是为了展示数据如何在不同的设备之间传递。

## [**计算设备**]

我们可以指定用于存储和计算的设备，如CPU和GPU。
默认情况下，张量是在内存中创建的，然后使用CPU计算它。


In [2]:
import tensorflow as tf

print( tf.device('/CPU:0') )
print( tf.device('/GPU:0') )
print( tf.device('/GPU:1') )# 尝试放到第二块GPU上 不会报错

## 查询可用gpu的数量


In [3]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

现在我们定义了两个方便的函数，
[**这两个函数允许我们在不存在所需所有GPU的情况下运行代码。**]


In [4]:
def try_gpu(i=0):  #@save 给一个GPU index 从0开始
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

print( try_gpu() )
print( try_gpu(10) )
print( try_all_gpus() )

## 张量与GPU

我们可以[**查询张量所在的设备。**]
默认情况下，张量是在CPU上创建的。


In [5]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

需要注意的是，无论何时我们要对多个项进行操作，
它们都必须在同一个设备上。
例如，如果我们对两个张量求和，
我们需要确保两个张量都位于同一个设备上，
否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

### [**存储在GPU上**] with try_gpu():

有几种方法可以在GPU上存储张量。
例如，我们可以在创建张量时指定存储设备。接
下来，我们在第一个`gpu`上创建张量变量`X`。
在GPU上创建的张量只消耗这个GPU的显存。
我们可以使用`nvidia-smi`命令查看显存使用情况。
一般来说，我们需要确保不创建超过GPU显存限制的数据。


In [6]:
with try_gpu():
    X = tf.ones((2, 3))
X.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [7]:
with try_gpu(2):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.20249474, 0.23604214, 0.47420204],
       [0.15948844, 0.36519194, 0.63190866]], dtype=float32)>

### 复制

如果我们[**要计算`X + Y`，我们需要决定在哪里执行这个操作**]。
我们可以将`X`传输到第二个GPU并在那里执行操作。
*不要*简单地`X`加上`Y`，因为这会导致异常，
运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。
由于`Y`位于第二个GPU上，所以我们需要将`X`移到那里，
然后才能执行相加运算。

In [8]:
with try_gpu(0):
    Z = X

In [9]:
print(X.device)
print(Y.device)
print(Z.device)

/job:localhost/replica:0/task:0/device:GPU:0
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


## 不同device上 可以直接相加！！！！

In [10]:
print(Y)
print(Z)

tf.Tensor(
[[0.20249474 0.23604214 0.47420204]
 [0.15948844 0.36519194 0.63190866]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [11]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.2024947, 1.2360421, 1.474202 ],
       [1.1594884, 1.3651919, 1.6319087]], dtype=float32)>

## Move data from CPU to GPU
## Move data from GPU to CPU
## with device:
### tf.device(f'/GPU:{i}') tf.device('/CPU:0')

In [12]:
tf.debugging.set_log_device_placement(True)

In [13]:
with tf.device('/CPU:0'):
    X = tf.Variable(X)

In [14]:
X.device

'/job:localhost/replica:0/task:0/device:CPU:0'

假设变量Z已经存在于第二个GPU上。 如果我们仍然在同一个设备作用域下调用Z2 = Z会发生什么？ 它将返回Z，而不会复制并分配新内存。

In [15]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True



## **神经网络与GPU**

类似地，神经网络模型可以指定设备。
下面的代码将模型参数放在GPU上。


In [16]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


在接下来的几章中，
我们将看到更多关于如何在GPU上运行模型的例子，
因为它们将变得更加计算密集。

当输入为GPU上的张量时，模型将在同一GPU上计算结果。


In [17]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-1.8950622],
       [-1.8950622]], dtype=float32)>

让我们(**确认模型参数存储在同一个GPU上。**)


In [18]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')